In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import silhouette_score
import seaborn as sns
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, losses
from sklearn.feature_selection import SelectKBest, f_classif
import warnings
from sklearn.exceptions import DataConversionWarning
with warnings.catch_warnings():
    warnings.filterwarnings(action='ignore', category=DataConversionWarning)
import random
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from collections import Counter
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import csv
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from F02_autoencoder_model_optimization_final import *
from F03_Feature_importance_functions import *
RANDOM_STATE = 55 ## We will pass it to every sklearn call so we ensure reproducibility

In [ ]:
# Load the datasets using pandas
df = pd.read_csv("M01_output_data.csv")


In [ ]:
df.shape

In [ ]:
df.iloc[:, :]

In [ ]:
cols_X_prot = slice(col_start, col_stop)
cols_X_met = slice(col_start2, col_stop2)
cols_clin = slice(col_start3, col_stop3)
cols_X_expr = slice(col_start4, col_stop4)
y_label = 'y_column_name'
n_all_feat = cols_X_met.stop
n_all_feat

In [ ]:
# Perform PCA on the data set to assess the degree of separation between clusters on the initial data
X = df.iloc[:, 0:n_all_feat]
y = df[y_label]
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X)
# Calculate Silhouette Score
pca_silhouette_score_initial = silhouette_score(X_pca, y)


In [ ]:
#### Apply pre-set parameters for feature selection and extraction derived from 
## previous optimization
# Determine total number of protein features
protn = cols_X_prot.stop - cols_X_prot.start
# Set k to select the top 12% of protein features (based on prior optimization results)
kprot = round(protn * 0.1237)

# Determine total number of metabolite features
metn = cols_X_met.stop - cols_X_met.start
# Set k to select the top 15% of metabolite features (based on prior optimization results)
kmet = round(metn * 0.1512)

# Set the number of units in the latent layer of the autoencoder (based on prior optimization results)
latent = round(cols_clin.stop * 0.000576)


In [ ]:
print(kprot, kmet, latent)

In [ ]:
####### Feature selection #########################################################
X_new_prot, df_selected_prot = feature_selection(df, kprot, cols_X_prot, y_label) ## select proteomic features
X_new_met, df_selected_met = feature_selection(df, kmet, cols_X_met, y_label) ## select metabolomic features

## assemble a dataframe containing only the selected features
df_selected = pd.concat([df_selected_prot, 
                         df_selected_met, df.iloc[:, cols_clin], df[y_label]], axis=1)


In [ ]:
####### Feature extraction #########################################################

# Define the autoencoder model architecture
n_feat = df_selected.iloc[:, :-1].shape[1]
n2 = round(n_feat/1.5)
if (n2 <= 70):
    n3 = round(n2/2)
if (n2 > 70) and (n2 <= 200):
    n3 = round(n2/5)
else:
    n3 = round(n2/6)


# Separate features and labels
X = df_selected.iloc[:, :-1]
y = df_selected.iloc[:, -1]


# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, 
                                                  test_size=0.25, random_state=42)


### run the autoencoder first with n_epochs set to 300, then use min_val_loss_epochs for the final run
## verbose can be either set to 1 to visualize the losses as the model runs, or 0 to silence them

history, val_loss, min_val_loss_epoch, bottleneck_features, encoder = ae_model_setup_and_run(n_feat, n2, n3, latent, X, X_train, X_val, 300, verbose = 0)
history, val_loss, min_val_loss_epoch, bottleneck_features, encoder = ae_model_setup_and_run(n_feat, n2, n3, latent, X, X_train, X_val, min_val_loss_epoch, verbose = 0)                                                                                       

# Create a pandas DataFrame for the extracted bottleneck features
extracted_features_df = pd.DataFrame(bottleneck_features, columns=[f"Feature_{i}" for i in range(latent)])
extracted_features_df[y_label] = y



In [ ]:
# Perform PCA on the extracted features data set to assess the degree of separation between clusters 
## after feature extraction

# Separate features and labels
X = bottleneck_features

# Perform PCA and t-SNE
pca = PCA(n_components=2)
X_pca_extr = pca.fit_transform(X)

# Calculate Silhouette Score
pca_silhouette_score_extracted_feat = silhouette_score(X_pca_extr, y)


In [ ]:
### plot the PCA of the initial data set and of the extracted feature 
## data set to visualize and compare the degree of separation


# This line sets the default global font size
plt.rcParams.update({'font.size': 16})

# Create a gridspec to partition our figure
gs = GridSpec(1, 2, width_ratios=[4, 0.8]) 

fig = plt.figure(figsize=(6.5, 4.5)) # Adjust as necessary

ax = fig.add_subplot(gs[0])

# Plot PCA embeddings
scatter = ax.scatter(X_pca[:, 0], X_pca[:, 1], c=y, cmap='viridis')

# Adding the legend in the space of the second subplot
ax_legend = fig.add_subplot(gs[1])
ax_legend.axis('off')
scatter_legend = ax_legend.legend(*scatter.legend_elements(), title='Classes', loc='center')

ax.set_xlabel('PC1')
ax.set_ylabel('PC2')
ax.set_title('PCA on initial data')

plt.savefig("PCA_initial_data.pdf", format='pdf', dpi=300) # This will save the plot as a PDF
print(f'Silhouette Score PCA on initial data: {pca_silhouette_score_initial}')


fig = plt.figure(figsize=(6.5, 4.5)) # Adjust as necessary

ax = fig.add_subplot(gs[0])

scatter = ax.scatter(X_pca_extr[:, 0], X_pca_extr[:, 1], c=y, cmap='viridis')
# Adding the legend in the space of the second subplot
ax_legend = fig.add_subplot(gs[1])
ax_legend.axis('off')
scatter_legend = ax_legend.legend(*scatter.legend_elements(), title='Classes', loc='center')

ax.set_xlabel('PC1')
ax.set_ylabel('PC2')
ax.set_title('PCA on extracted features')
plt.savefig("PCA_extracted_features.pdf", format='pdf', dpi=300) # This will save the plot as a PDF

print(f'Silhouette Score PCA on extracted features: {pca_silhouette_score_extracted_feat}')

In [ ]:
#### Compute importance scores for the original features relative to the latent layer

X = df_selected.iloc[:, :-1]

importance_scores = feature_importance_latent(encoder, X, eps=0.5)

In [ ]:
### Identify the indices of the important features in the original dataframe
### Also, calculate the threshold values to define important features
## the threshold values are calculated as the 70th percentile of the set of importance values
# for each neuron in the latent layer

top_indices, thresholds = top_percentile_indices(importance_scores)

In [ ]:
### create a set of dataframes with the important values for each neuron in the latent layer
### normalize the importance values in the range between 0 to 1
### this data will be used to plot the distribution of importance scores

dfs, scaler = importance_scores_scaling(importance_scores, thresholds)

In [ ]:
# Generate density plots of the importance scores 
plot_density(dfs, thresholds, scaler)

In [ ]:
#### Create dataframes with the selected 
## important features for each neuron in the latent layer
# top_indices is a list of lists. 
# Each sublist is a list of top indices for each neuron in the latent layer.

df_selected_list = []

for indices in top_indices:
    df_selected_list.append(df_selected.iloc[:, indices])

In [ ]:
#### Save list of important features to txt files that will be used for the MetaboAnalyst analysis
## (Method 4)
for i, df in enumerate(df_selected_list):
    with open(f"module_{i+1}.txt", "w") as file:
        for col in df.columns:
            file.write(col + '\n')
